<a href="https://colab.research.google.com/github/HausReport/MightyLogic/blob/master/Notebooks/Turf_War_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up environment


In [6]:

import sys
# need newer plotly/plotly_express than collab provides?
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly"
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly_express"

#
# Moving some of the guts to GitHub and refactoring 
#
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/MightyLogic.git#egg=MightyLogic"

#
# Imports
#
from random import randrange
from scipy.signal import convolve2d
from enum import Enum, auto
from pprint import pprint
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math

#import sklearn
#from sklearn.preprocessing import PowerTransformer, QuantileTransformer

  Cloning https://github.com/HausReport/MightyLogic.git to /tmp/pip-install-zrbsfuen/mightylogic_cefef6b2b9a44d19982367ffcfbbcc8c
  Running command git clone -q https://github.com/HausReport/MightyLogic.git /tmp/pip-install-zrbsfuen/mightylogic_cefef6b2b9a44d19982367ffcfbbcc8c
  Created wheel for MightyLogic: filename=MightyLogic-0.1-py3-none-any.whl size=6853 sha256=75b549355eb433b8361561360695febcd66593322406d097fb548522dbed41c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-sgrt3wzw/wheels/84/7a/55/d74e49b550ce0fddb2379db726853920650df706dedea8daae
Successfully built MightyLogic
  Attempting uninstall: MightyLogic
    Found existing installation: MightyLogic 0.1
    Uninstalling MightyLogic-0.1:
      Successfully uninstalled MightyLogic-0.1


In [7]:

from MightyLogic.Currency import Currency as Currency
from MightyLogic.Values import Values as Values

Values.GOLD

0.6

# Turf War Demo

In [8]:
from MightyLogic.TurfWar.Tiles.BuildingFactory import *
from MightyLogic.TurfWar.TurfWarMap import TurfWarMap
#
# A Turf War Map
#

map2 =TurfWarMap()

# Use convenience functions to build value array
map2.addTile('A',1, mountains(3000,2000))
map2.addTile('A',2, forest(3000,0,2000))
map2.addTile('A',3, plains(3000, 2000))
map2.addTile('A',4, mountains(1600,1000))
map2.addTile('A',5, forest(3000,0,2000))
map2.addTile('A',6, plains(3000, 2000))

map2.addTile('B',1, factory(1650,500,5000))
map2.addTile('B',2, mountains(5000,5000))
map2.addTile('B',3, lighthouse(3500,300,7000))
map2.addTile('B',4, plains(5000,5000))
map2.addTile('B',5, mountains(5000,5000))
map2.addTile('B',6, gold_mine(90000,500,5000))

map2.addTile('C',1, forest(3000,0,2000))
map2.addTile('C',2, plains(5000,5000))
map2.addTile('C',3, mountains(5000,5000))
map2.addTile('C',4, forest(5000,0,5000))
map2.addTile('C',5, hells_gate(600,800,8500))
map2.addTile('C',6, mountains(3000,2000))

map2.addTile('D',1, mountains(3000,2000))
map2.addTile('D',2, witch_hut(3500,600, 7000))
map2.addTile('D',3, plains(5000,5000))
map2.addTile('D',4, altar(600,800,8500))
map2.addTile('D',5, forest(5000,0,5000))
map2.addTile('D',6, lighthouse(2000, 400, 5000))

map2.addTile('E',1, plains(3000, 2000))
map2.addTile('E',2, mountains(3000,2000))
map2.addTile('E',3, forest(3000,0,2000))
map2.addTile('E',4, plains(3000, 2000))
map2.addTile('E',5, mountains(3000,2000))
map2.addTile('E',6, forest(3000,0,2000))


# Payout array
image = map2.getValues()
pprint(image)

[[3000, 3000, 3000, 1600, 3000, 3000],
 [17516.5, 5000, 18503.5, 5000, 5000, 26500.0],
 [3000, 5000, 5000, 5000, 29030.0, 3000],
 [3000, 23003.5, 5000, 29030.0, 5000, 16002.0],
 [3000, 3000, 3000, 3000, 3000, 3000]]


In [9]:
# NSEW Averaging gives strategic value of a position
# I think of it as an image blur, but iirc it's equivalent to a discrete gradient operator
filter_kernel = [[0, .2, 0],
                 [.2, .2, .2],
                 [0, .2, 0]]

res = convolve2d(image, 
                 filter_kernel,
                 mode='same', 
                 boundary='fill', 
                 fillvalue=0)
pprint(res)

array([[ 4703.3,  2800. ,  5220.7,  2520. ,  2520. ,  6500. ],
       [ 5703.3,  9804. ,  7300.7,  7020.7, 13706. ,  7500. ],
       [ 5703.3,  8200.7,  7700.7, 14612. ,  9406. , 14906.4],
       [ 6400.7,  7800.7, 13006.7,  9406. , 16412.4,  5400.4],
       [ 1800. ,  6400.7,  2800. ,  7606. ,  2800. ,  4400.4]])


In [10]:
#
# An approach that shows well-labeled heatmaps.
#

def show_heatmap(arr, title):
  flip = np.flip( np.array(arr), axis=0)  # use numpy to flip array vertically
  z = flip.round()

  x = ['1', '2', '3', '4','5','6']
  y = ['E','D','C','B','A']

  #z_text = ["%.0f" % number for number in z] - doesn't work

  z_text = flip.round( decimals=-1)

  # Possible to build up z_text to make it more friendly
  # z_text = [['Win', 'Lose', 'Win'],
  #          ['Lose', 'Lose', 'Win'],
  #          ['Win', 'Win', 'Lose']]

  fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')
  fig.update_layout(title=title, autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
  fig.show()



In [11]:
show_heatmap(image,"Payout Values for 3rd Rounds")
show_heatmap(res,"Strategic Values for Non-Payout Rounds")

In [12]:
fig = go.Figure(data=[go.Surface(z=image)])
fig.update_layout(title='Payout Value of Positions', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [13]:
fig = go.Figure(data=[go.Surface(z=res)])
fig.update_layout(title='Strategic Value of Positions', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [15]:
payout = map2.getDataFrame()
payout

,1,2,3,4,5,6
A,3000.0,3000.0,3000.0,1600.0,3000.0,3000.0
B,17516.5,5000.0,18503.5,5000.0,5000.0,26500.0
C,3000.0,5000.0,5000.0,5000.0,29030.0,3000.0
D,3000.0,23003.5,5000.0,29030.0,5000.0,16002.0
E,3000.0,3000.0,3000.0,3000.0,3000.0,3000.0


In [14]:
# Puts data into super-clear dataframe, might use elsewhere.
oneTurn = map2.getStrategicDataFrame()
oneTurn

,1,2,3,4,5,6
A,4703.3,2800.0,5220.7,2520.0,2520.0,6500.0
B,5703.3,9804.0,7300.7,7020.7,13706.0,7500.0
C,5703.3,8200.7,7700.7,14612.0,9406.0,14906.4
D,6400.7,7800.7,13006.7,9406.0,16412.4,5400.4
E,1800.0,6400.7,2800.0,7606.0,2800.0,4400.4


In [22]:
map2.grid_distance(0,0,2,2)

#grid_distance(0,0,2,2) == grid_distance(2,2,0,0)
#payout.iloc[row_dict['E']][6]

AttributeError: ignored

In [17]:
row_dict = { 'A': 0, 'B':1, 'C':2, 'D':3, 'E':4}
rows = [ 'A', 'B', 'C', 'D', 'E']
cols = [1,2,3,4,5,6]

for r in rows:
  for c in cols:
    print(f"({r}-{c})",end='')
  print("")

for r in rows:
  for c in cols:
    print( "("+str(payout.iloc[row_dict[r]][c])+")",end='')
  print("")

(A-1)(A-2)(A-3)(A-4)(A-5)(A-6)
(B-1)(B-2)(B-3)(B-4)(B-5)(B-6)
(C-1)(C-2)(C-3)(C-4)(C-5)(C-6)
(D-1)(D-2)(D-3)(D-4)(D-5)(D-6)
(E-1)(E-2)(E-3)(E-4)(E-5)(E-6)
(3000.0)(3000.0)(3000.0)(1600.0)(3000.0)(3000.0)
(17516.5)(5000.0)(18503.5)(5000.0)(5000.0)(26500.0)
(3000.0)(5000.0)(5000.0)(5000.0)(29030.0)(3000.0)
(3000.0)(23003.5)(5000.0)(29030.0)(5000.0)(16002.0)
(3000.0)(3000.0)(3000.0)(3000.0)(3000.0)(3000.0)


In [19]:
#
# Shows the maximum possible single tile you can take
# at a given turn.
#

payout = map2.getDataFrame()



show_heatmap(image,"Strategic Positions for Turns 3n")

foo = map2.getStrategicDataFrame()
show_heatmap(foo,"Strategic Positions for Turns 3n+2")

#thisThing = pd.DataFrame(foo, index=["A","B","C","D","E"], columns=[1,2,3,4,5,6])
#bar = oneMove(thisThing)
#show_heatmap(bar,"Strategic Positions for Turns 3n+1")




In [20]:
#
# Shows payout shares for holding a tile on round 3n 
#

gm = map2.getTile('C',5) #b6 # c5 # gold_mine(90000,500,5000)
print(gm.payouts())


```Payouts for C:5 Hell's Gate
Place               Reward
First               42 soul dust, 595 legendary souls, 56 contribution
2nd                 36 soul dust, 510 legendary souls, 48 contribution
3rd                 30 soul dust, 425 legendary souls, 40 contribution
4-6                 24 soul dust, 340 legendary souls, 32 contribution
7-11                18 soul dust, 255 legendary souls, 24 contribution
12-25               12 soul dust, 170 legendary souls, 16 contribution
26-44               6 soul dust, 85 legendary souls, 8 contribution
```



In [21]:
pprint(map2.arr)

{('A', 1): Tile(name=Mountains, row=A, column=1),
 ('A', 2): Tile(name=Forest, row=A, column=2),
 ('A', 3): Tile(name=Plains, row=A, column=3),
 ('A', 4): Tile(name=Mountains, row=A, column=4),
 ('A', 5): Tile(name=Forest, row=A, column=5),
 ('A', 6): Tile(name=Plains, row=A, column=6),
 ('B', 1): Tile(name=Factory, row=B, column=1),
 ('B', 2): Tile(name=Mountains, row=B, column=2),
 ('B', 3): Tile(name=Lighthouse, row=B, column=3),
 ('B', 4): Tile(name=Plains, row=B, column=4),
 ('B', 5): Tile(name=Mountains, row=B, column=5),
 ('B', 6): Tile(name=Gold Mine, row=B, column=6),
 ('C', 1): Tile(name=Forest, row=C, column=1),
 ('C', 2): Tile(name=Plains, row=C, column=2),
 ('C', 3): Tile(name=Mountains, row=C, column=3),
 ('C', 4): Tile(name=Forest, row=C, column=4),
 ('C', 5): Tile(name=Hell's Gate, row=C, column=5),
 ('C', 6): Tile(name=Mountains, row=C, column=6),
 ('D', 1): Tile(name=Mountains, row=D, column=1),
 ('D', 2): Tile(name=Witch Hut, row=D, column=2),
 ('D', 3): Tile(name=Pl

# Notes


1.   Annotated heatmap: https://plotly.com/python/annotated-heatmap/
2.   A-Star search on grid in python at https://github.com/BaijayantaRoy/Medium-Article/blob/master/A_Star.ipynb

